# SQL-Based Exploratory Analysis of Credit Risk

This notebook explores a cleaned credit risk dataset using SQL queries. The data has been preprocessed to remove outliers and engineered to include features such as income brackets, age groups, debt ratio categories, and credit utilization bands.

The goal is to use SQLite and SQL queries executed via Python to answer key questions related to borrower delinquency.


## Setup

In [5]:
import pandas as pd
import sqlite3

df = pd.read_csv('../data/clean_credit_risk.csv')
connection_object = sqlite3.connect('../database/credit_risk.db')
df.to_sql('credit_risk', connection_object, if_exists='replace', index=False)

16182

## Analysis

### What share of borrowers defaulted?

In [7]:
print("Q1: Overall Default Rate")
query = """
SELECT AVG(dlq_2yrs) AS default_rate
FROM credit_risk;
"""
display(pd.read_sql_query(query, conn))

Q1: Overall Default Rate


,default_rate
0,0.496539
